In [3]:
import gradio as gr

In [8]:
# 簡単なインターフェースの実装
def text2text(text):
    text = "<<" + text + ">>"
    return text

input_text = gr.Text()
output_text = gr.Text()

demo = gr.Interface(inputs=input_text, outputs=output_text, fn=text2text)
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [11]:
# 複数機能の実装
def text2text_rich(text):
    top = "^" * len(text)
    bottom = "v" * len(text)
    text = f" {top}\n<{text}>\n {bottom}"
    return text

with gr.Blocks() as demo:
    input_text = gr.Text()
    button1 = gr.Button(value = "Normal")
    button2 = gr.Button(value = "Rich")
    output_text = gr.Text()

    button1.click(inputs=input_text, outputs=output_text, fn=text2text)
    button2.click(inputs=input_text, outputs=output_text, fn=text2text_rich)
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
